In [1]:
# imports
# Reference: https://www.cs.toronto.edu/~vmnih/docs/dqn.pdf
import os
import time
import numpy as np
import tensorflow as tf
print(tf.__version__)
import gym
import random
from collections import deque
import matplotlib.pyplot as plt
# choose a GPU card
os.environ['CUDA_VISIBLE_DEVICES']="2"

2.0.0


ModuleNotFoundError: No module named 'gym'

In [ ]:
env = gym.make("BreakoutNoFrameskip-v4")
obs = env.reset()
obs.shape

In [ ]:
def preprocess(image):
    """ prepro 210x160x3 uint8 frame into 6400 (80x80) 2D float array """
    image = image[35:195] # crop
    image = image[::2,::2,0] # downsample by factor of 2
    image[image == 144] = 0 # erase background (background type 1)
    image[image == 109] = 0 # erase background (background type 2)
    image[image != 0] = 1 # everything else just set to 1
    return np.reshape(image.astype(np.float).ravel(), [80,80])

## Define Q network

In [ ]:
# This is now exactly the same as Ghani's CNN settings
# need to adjust to 80**)
learning_rate=0.001
state_size=[80,80,1]
action_size=2
n_outputs=2 # we are only using action 2 and 3 (RIGHT and LEFT)
#
input_height=80
input_width=80
input_channels=1

In [ ]:
# Define a Q network with input size of [80,80], and an output size of size of action space 2
# Note that the action space is 4 with following meanings:
# ['NOOP', 'FIRE', 'RIGHT', 'LEFT']
# We will only be taking action 2 or 3
class DQN(object):
    def __init__(self, state_size, action_size, learning_rate, name=''):
        self.state_size = state_size
        self.action_size = action_size
        self.learning_rate = learning_rate

        with tf.variable_scope(name):
            with tf.name_scope("inputs"):
                # Jieda note: state is preprocessd 80*80 array
                self.inputs = tf.placeholder(tf.float32, [None,*state_size], name = "inputs")

            with tf.name_scope("conv1"):
                self.conv1 = tf.layers.conv2d(
                inputs=self.inputs, filters=32, kernel_size=[8, 8], strides=4,
                kernel_initializer=tf.variance_scaling_initializer(scale=2),
                padding="VALID", activation=tf.nn.relu, use_bias=False, name='conv1')
                
                self.conv1_out = tf.nn.relu(self.conv1, name='conv1_out')
             
            with tf.name_scope("conv2"):
                
                self.conv2 = tf.layers.conv2d(
                inputs = self.conv1_out, filters=64,
                kernel_size=[4,4], strides=[2,2],padding="VALID",
                kernel_initializer = tf.variance_scaling_initializer(scale=2),
                activation=tf.nn.relu, use_bias=False, name='conv2')
                
                self.conv2_out = tf.nn.relu(self.conv2, name='conv2_out')
            
            with tf.name_scope("conv3"):
            
                self.conv3 = tf.layers.conv2d(
                inputs = self.conv2_out, filters=64,
                 kernel_size=[3,3], strides=[1,1], padding="VALID",
                 kernel_initializer = tf.variance_scaling_initializer(scale=2),
                 name = "conv3")
                
                self.conv3_out = tf.nn.relu(self.conv3, name='conv3_out')
            
            with tf.name_scope("flatten"):
                self.flatten = tf.contrib.layers.flatten(self.conv3_out)
                

            with tf.name_scope("fc1"):
                self.fc1 = tf.layers.dense(inputs=self.flatten,
                                          units = 512, activation = tf.nn.relu,
                                          kernel_initializer = tf.contrib.layers.xavier_initializer(), name = "fc1")
            
            with tf.name_scope("fc1"):
                self.fc2 = tf.layers.dense(inputs=self.flatten,
                                          units = 512, activation = tf.nn.relu,
                                          kernel_initializer = tf.contrib.layers.xavier_initializer(), name = "fc2")

            with tf.name_scope("outputs"):
                self.outputs = tf.layers.dense(inputs = self.fc2,
                                              units = action_size,
                                              kernel_initializer = tf.contrib.layers.xavier_initializer(),
                                              activation = None)
            # Output is the approximated Action Values Q(s,a), so we don't need any activation
            
    def get_outputs(self):

        return self.outputs

    def get_weights(self):

        # give all the weights of that network
        trainable_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=self.name)
        # create a dictionary to contain the values of the network weights
        trainable_vars_by_name = {var.name[len(scope.name):]: var
                                 for var in trainable_vars}

        return trainable_vars_by_name
        


#             with tf.name_scope("loss"):
#                 self.cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = self.logits, labels =self.actions)
#                 # Jieda noted: for baseline, we subtract the value_estimate_ from discounted_episode_rewards
#                 self.weighted_negative_likelihoods = tf.multiply(self.cross_entropy, self.discounted_episode_rewards)
#                 self.loss = tf.reduce_mean(self.weighted_negative_likelihoods)

#             with tf.name_scope("train"):
#                 self.optimizer = tf.train.RMSPropOptimizer(self.learning_rate)
#                 self.train_opt = self.optimizer.minimize(self.loss)

tf.reset_default_graph()
X_state=tf.placeholder(tf.float32,shape=[None,input_height, input_width,input_channels])
# initialize the two Q network
online_q = DQN(state_size, action_size, learning_rate, 'online_q_network')
target_q = DQN(state_size, action_size, learning_rate, 'target_q_network')
# get the output and weights from online q network
online_q_values=online_q.get_outputs()
online_q_weights=online_q.get_weights()
# get the output and weights from target q network
target_q_values=target_q.get_outputs()
target_q_weights=target_q.get_weights()
# # copy the weights of online network to target network
# copy_ops = [target_q_wegiths.assign(online_q_weights[var_name]) for var_name, target_var in target_vars.items()]
# #
# copy_online_to_target = tf.group(*copy_ops)

In [ ]:
## Learning parameters

In [ ]:
learning_rate=0.001
# Use adam optimizer
beta_1=0.9
beta_2=0.999
epsilon=1e-07

In [ ]:
## Replay Buffer

In [ ]:
replay_memory_size=500000
replay_memory = deque([],maxlen=replay_memory_size)

In [ ]:
class ReplayBuffer:
    def __init__(self,maxlen):
        self.maxlen=maxlen
        self.buf = 

def draw_samples_from_rb(batch_size):
    

In [ ]:
## Epsilon Greedy

In [ ]:
eps_min = 0.1
eps_max = 1.0
eps_decay_steps = 2000000
def epsilon_greedy(q_values, step):
    # Note: we gradually decrease epsilon, we explore more in the beginning, less towards later
    epsilon = max(eps_min, eps_max-(eps_max-eps_min)*step/eps_decay_steps)
    if np.random.rand()<epsilon:
        return np.random.randint(n_outputs)
    else:
        return np.argmax(q_values)

In [ ]:
## Training parameters

In [ ]:
n_steps = 4000000
# In the very beginning, First have 10000 samples in the replay buffer
training_start=10000
# Before each training, we run 10 episodes and add those samples to replay buffer
training_interval=4
save_steps=1000
copy_steps = 10000 # copy the online network parameters to target network every 10000 steps
discount_rate=0.99
skip_start=90
batch_size=50
iteration=0
done=True